In [2]:
import moses
import selfies as sf
import os
import numpy as np
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from rdkit import Chem
from tqdm import tqdm
from moses import CharVocab, StringDataset

tokenizer = AutoTokenizer.from_pretrained("zjunlp/MolGen-large-opt")
model = AutoModelForSeq2SeqLM.from_pretrained("zjunlp/MolGen-large-opt")

test = moses.get_dataset('test')
vocab = CharVocab.from_data(test)

if not os.path.exists('results'): os.mkdir('results')

def external_evaluation(testDataset):
    # Evaluation with external dataset
    new_mol = []
    for i, mol in tqdm(enumerate(testDataset)):
        sf_input = tokenizer(sf.encoder(mol), return_tensors="pt")
        # beam search
        molecules = model.generate(input_ids=sf_input["input_ids"],
                                      attention_mask=sf_input["attention_mask"],
                                      max_length=65,
                                      min_length=5,
                                      num_return_sequences=10,
                                      num_beams=20)
        sf_output = [sf.decoder(tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True).replace(" ","")) for g in molecules]
        np.save(f'results/smiles_{i}.npy', np.array(sf_output))
    
    

external_evaluation(test[50000:])

/home/marc/anaconda3/envs/imageDL/lib/python3.8/site-packages/moses/metrics/utils.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  _mcf.append(_pains, sort=True)['smarts'].values]


In [15]:
generated_smiles = np.array([np.load('results/'+smile) for smile in os.listdir('results/')]).ravel()

df =pd.DataFrame(moses.get_all_metrics(generated_smiles, test =test[50000:50000+len(os.listdir('results/'))]), index = ['results'])
df

,valid,unique@1000,unique@10000,FCD/Test,SNN/Test,Frag/Test,Scaf/Test,FCD/TestSF,SNN/TestSF,Frag/TestSF,Scaf/TestSF,IntDiv,IntDiv2,Filters,logP,SA,QED,weight,Novelty
results,1.0,0.919,0.9029,3.568158,0.634007,0.98392,0.941199,5.516483,0.46951,0.980654,0.012345,0.874747,0.867003,0.872051,0.64762,0.479579,0.106772,32.440149,0.971243
